# 整合視覺化測試筆記本
# Integrated Visualization Test Notebook

此筆記本旨在測試和展示 KEEN 函式庫中針對不同數據類型（`.int` 和 `.dat`/CITS）的整合視覺化功能。
This notebook aims to test and demonstrate the integrated visualization capabilities of the KEEN library for different data types (`.int` and `.dat`/CITS).

**主要測試內容 / Main Test Contents:**
1.  從 `.txt` 檔案載入實驗會話 / Load experiment session from `.txt` file.
2.  處理 `.int` 檔案 (拓撲數據) / Process `.int` file (topography data):
    *   顯示 2D 拓撲圖 / Display 2D topography map.
    *   提取並顯示高度剖面線 / Extract and display height profile.
3.  處理 `.dat` 檔案 (CITS 光譜數據) / Process `.dat` file (CITS spectroscopy data):
    *   顯示 CITS 偏壓切片 / Display CITS bias slice.
    *   顯示線剖面能帶圖 / Display line profile band diagram.
    *   顯示堆疊光譜圖 / Display stacked spectra plot.

## 1. 設定與初始化 / Setup and Initialization

In [ ]:
# ============== 設定區域 / Configuration Section ==============

# 檔案路徑設定 / File path configuration
TXT_FILE_PATH = "/Users/yangziliang/Git-Projects/keen/testfile/20250521_Janus Stacking SiO2_13K_113.txt"

# .int 檔案選擇 (用於拓撲和剖面線) / .int file selection (for topography and profile)
# 注意：如果指定的檔案不存在，程式會嘗試選擇第一個可用的 'TopoFwd.int' 或類似檔案
# Note: If the specified file doesn't exist, the program will try to select the first available 'TopoFwd.int' or similar file
INT_FILE_KEY_CONTAINS = "TopoFwd" # 通常是 '...TopoFwd.int'

# .dat (CITS) 檔案選擇 / .dat (CITS) file selection
# 注意：如果指定的檔案不存在，程式會自動選擇第一個可用的 CITS 檔案
# Note: If the specified file doesn't exist, the program will automatically select the first available CITS file

CITS_FILE_KEY = "Lia1R_Matrix"  # 完整的檔案鍵名 / Complete file key name

# .int 剖面線參數 / .int profile parameters
PROFILE_START_COORD = (20, 20)    # 剖面線起點 (像素) / Profile start point (pixels)
PROFILE_END_COORD = (100, 100)  # 剖面線終點 (像素) / Profile end point (pixels)

# CITS 分析參數 (與 cits_workflow_test.ipynb 保持一致以便比較) / CITS analysis parameters (consistent with cits_workflow_test.ipynb for comparison)
CITS_BIAS_INDEX = 50
CITS_LINE_START = (10, 10)
CITS_LINE_END = (50, 50)
CITS_OFFSET_FACTOR = 1.0
CITS_MAX_CURVES = 15
CITS_USE_LOG_SCALE = False

# ================================================================

print("⚙️ 設定完成 / Configuration completed")
print(f"📁 TXT 檔案: {TXT_FILE_PATH}")
print(f"🔍 .int 檔案關鍵字: {INT_FILE_KEY_CONTAINS}")
print(f"🔍 CITS 檔案: {CITS_FILE_KEY}")
print(f"📊 CITS 分析參數: 偏壓索引={CITS_BIAS_INDEX}, 線段={CITS_LINE_START}→{CITS_LINE_END}")

⚙️ 設定完成 / Configuration completed
📁 TXT 檔案: /Users/yangziliang/Git-Projects/keen/testfile/20250521_Janus Stacking SiO2_13K_113.txt
🔍 .int 檔案關鍵字: topofwd
🔍 CITS 檔案: 20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix
📊 CITS 分析參數: 偏壓索引=50, 線段=(10, 10)→(50, 50)


In [17]:
# 導入必要的模組 / Import necessary modules
import sys
import os
import numpy as np
import plotly.io as pio
pio.renderers.default = "vscode" # 在 VSCode 中顯示 Plotly 圖形 / To display Plotly figures in VSCode

# 將父目錄加入 sys.path 以便導入 keen 的核心模組
# Add parent directory to sys.path to import keen's core modules
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')) # 從 backend/test/notebooks 退回到 keen/
if module_path not in sys.path:
    sys.path.append(module_path)


from backend.core.experiment_session import ExperimentSession
from backend.core.visualization.spm_plots import SPMPlotting
from backend.core.visualization.spectroscopy_plots import SpectroscopyPlotting
from backend.core.analysis.cits_analysis import (
    extract_cits_bias_slice,
    extract_line_spectra_data,
    prepare_stacked_spectra_data
)
# from backend.core.analysis.spm_analysis import extract_line_profile # 假設此函數存在或將被創建

print("✅ 模組導入完成 / Modules imported successfully")

✅ 模組導入完成 / Modules imported successfully


## 2. 載入實驗數據 / Load Experiment Data

In [25]:
# 重新載入模組以確保使用最新版本
import importlib
import sys

# 重新載入相關模組
if 'backend.core.experiment_session' in sys.modules:
    importlib.reload(sys.modules['backend.core.experiment_session'])
    print("🔄 重新載入 ExperimentSession 模組")

from backend.core.experiment_session import ExperimentSession

# 刪除舊的 session 對象（如果存在）
if 'session' in locals():
    del session
    print("🗑️ 刪除舊的 session 對象")

print("📂 載入實驗會話...")
try:
    session = ExperimentSession(TXT_FILE_PATH)
    print(f"✅ 實驗: {session.experiment_name}")

    available_files_dict = session.available_files
    print("\n📋 可用檔案列表:")
    for file_type, file_keys in available_files_dict.items():
        if file_keys:
            print(f"  {file_type.upper()}:")
            for key in file_keys:
                print(f"    - {key}")
                
    # 測試新的 get_topo_files() 方法
    print("\n🔍 測試 get_topo_files() 方法:")
    topo_files = session.get_topo_files()
    print(f"  返回 {len(topo_files)} 個拓撲檔案")
    
except Exception as e:
    print(f"❌ 載入實驗會話失敗: {e}")
    raise

🔄 重新載入 ExperimentSession 模組
🗑️ 刪除舊的 session 對象
📂 載入實驗會話...
✅ 實驗: 20250521_Janus Stacking SiO2_13K_113

📋 可用檔案列表:
  TXT:
    - 20250521_Janus Stacking SiO2_13K_113
  INT:
    - 20250521_Janus Stacking SiO2_13K_113TopoFwd
    - 20250521_Janus Stacking SiO2_13K_113TopoBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1XFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1XBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1YFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1YBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1RFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1RBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia2RFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia2RBwd
    - 20250521_Janus Stacking SiO2_13K_113It_to_PCFwd
    - 20250521_Janus Stacking SiO2_13K_113It_to_PCBwd
  CITS:
    - 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
    - 20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix
    - 20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix
    - 20250521_Janus Stacking SiO2_13K

## 3. `.int` 檔案分析與繪圖 / `.int` File Analysis and Plotting

In [26]:
# 載入 .int 檔案 (拓撲數據) / Load .int file (topography data)
print(f"🔍 搜尋包含關鍵字 '{INT_FILE_KEY_CONTAINS}' 的 .int 檔案...")

int_file_proxy = None
selected_int_file_key = None

try:
    # 獲取所有可用的拓撲檔案
    topo_files = session.get_topo_files()
    print(f"📋 可用的拓撲檔案: {topo_files}")
    
    # 尋找包含指定關鍵字的檔案
    target_file_key = None
    for file_key in topo_files:
        if INT_FILE_KEY_CONTAINS.lower() in file_key.lower():
            target_file_key = file_key
            break
    
    # 如果找不到包含關鍵字的檔案，使用第一個可用的檔案
    if target_file_key is None and topo_files:
        target_file_key = topo_files[0]
        print(f"⚠️  未找到包含 '{INT_FILE_KEY_CONTAINS}' 的檔案，使用第一個可用檔案: {target_file_key}")
    elif target_file_key:
        print(f"✅ 找到匹配檔案: {target_file_key}")
    
    if target_file_key:
        # 載入檔案
        int_file_proxy = session[target_file_key]
        selected_int_file_key = target_file_key
        
        print(f"✅ .int 檔案載入成功: {selected_int_file_key}")
        print(f"  數據形狀: {int_file_proxy.data.image.shape}")
        print(f"  物理尺寸 (X): {int_file_proxy.data.x_range:.2f} nm")
        print(f"  物理尺寸 (Y): {int_file_proxy.data.y_range:.2f} nm")
        print(f"  像素尺度: {int_file_proxy.data.pixel_scale_x:.3f} nm/pixel")
    else:
        print("❌ 沒有找到任何拓撲檔案")
        
except Exception as e:
    print(f"❌ 載入 .int 檔案失敗: {e}")
    import traceback
    traceback.print_exc()
    int_file_proxy = None

# 檢查 int_file_proxy 是否成功載入
if int_file_proxy is None:
    print("🛑 無法繼續 .int 檔案的分析，因為沒有成功載入的 .int 檔案。")
else:
    print(f"👍 準備使用 '{selected_int_file_key}' 進行 .int 檔案分析。")

檔案長度 (1000000) 與預期不符 (262144)
解析 INT 檔案時出錯: cannot reshape array of size 250000 into shape (256,256)
檔案載入失敗: 20250521_Janus Stacking SiO2_13K_113TopoFwd, 錯誤: '>=' not supported between instances of 'int' and 'str'
解析 INT 檔案時出錯: cannot reshape array of size 250000 into shape (256,256)
檔案載入失敗: 20250521_Janus Stacking SiO2_13K_113TopoFwd, 錯誤: '>=' not supported between instances of 'int' and 'str'


🔍 搜尋包含關鍵字 'topofwd' 的 .int 檔案...
📋 可用的拓撲檔案: ['20250521_Janus Stacking SiO2_13K_113TopoFwd', '20250521_Janus Stacking SiO2_13K_113TopoBwd', '20250521_Janus Stacking SiO2_13K_113Lia1XFwd', '20250521_Janus Stacking SiO2_13K_113Lia1XBwd', '20250521_Janus Stacking SiO2_13K_113Lia1YFwd', '20250521_Janus Stacking SiO2_13K_113Lia1YBwd', '20250521_Janus Stacking SiO2_13K_113Lia1RFwd', '20250521_Janus Stacking SiO2_13K_113Lia1RBwd', '20250521_Janus Stacking SiO2_13K_113Lia2RFwd', '20250521_Janus Stacking SiO2_13K_113Lia2RBwd', '20250521_Janus Stacking SiO2_13K_113It_to_PCFwd', '20250521_Janus Stacking SiO2_13K_113It_to_PCBwd']
✅ 找到匹配檔案: 20250521_Janus Stacking SiO2_13K_113TopoFwd
✅ .int 檔案載入成功: 20250521_Janus Stacking SiO2_13K_113TopoFwd
❌ 載入 .int 檔案失敗: Failed to load file '20250521_Janus Stacking SiO2_13K_113TopoFwd': ["Failed to load file: '>=' not supported between instances of 'int' and 'str'"]
🛑 無法繼續 .int 檔案的分析，因為沒有成功載入的 .int 檔案。


Traceback (most recent call last):
  File "/var/folders/s8/x7qjkqzs44lg2tdb36y2xh680000gn/T/ipykernel_74516/2993195230.py", line 32, in <module>
    print(f"  數據形狀: {int_file_proxy.data.image.shape}")
                         ^^^^^^^^^^^^^^^^^^^
  File "/Users/yangziliang/Git-Projects/keen/backend/core/file_proxy.py", line 63, in data
    raise RuntimeError(f"Failed to load file '{self._file_key}': {result.errors}")
RuntimeError: Failed to load file '20250521_Janus Stacking SiO2_13K_113TopoFwd': ["Failed to load file: '>=' not supported between instances of 'int' and 'str'"]


In [ ]:
# 調試：檢查 available_files 結構
print("🔍 調試：available_files 字典內容：")
for key, value in session.available_files.items():
    print(f"  {key}: {value}")

print("\n🔍 調試：get_topo_files() 方法結果：")
topo_files = session.get_topo_files()
print(f"  {topo_files}")

print("\n🔍 調試：直接查看 available_files['int']：")
int_files = session.available_files.get('int', [])
print(f"  {int_files}")

# 測試所有新添加的便利方法
print("🧪 測試新添加的便利方法:")
print("=" * 50)

print("\n📋 get_topo_files() 方法:")
topo_files = session.get_topo_files()
print(f"  返回 {len(topo_files)} 個拓撲檔案")
print(f"  前3個檔案: {topo_files[:3]}")

print("\n📋 get_cits_files() 方法:")
cits_files = session.get_cits_files()
print(f"  返回 {len(cits_files)} 個 CITS 檔案")
print(f"  所有檔案: {cits_files}")

print("\n📋 get_sts_files() 方法:")
sts_files = session.get_sts_files()
print(f"  返回 {len(sts_files)} 個 STS 檔案")
print(f"  所有檔案: {sts_files}")

print("\n📋 get_txt_files() 方法:")
txt_files = session.get_txt_files()
print(f"  返回 {len(txt_files)} 個 TXT 檔案")
print(f"  所有檔案: {txt_files}")

print("\n✅ 所有便利方法測試完成！")
print("\n🔍 available_files 字典內容（用於比較）:")
for key, value in session.available_files.items():
    print(f"  {key}: {len(value)} 個檔案")
    
print("\n✅ 修復驗證: 所有 get_*_files() 方法都正常工作！")

🔍 調試：available_files 字典內容：
  txt: ['20250521_Janus Stacking SiO2_13K_113']
  int: ['20250521_Janus Stacking SiO2_13K_113TopoFwd', '20250521_Janus Stacking SiO2_13K_113TopoBwd', '20250521_Janus Stacking SiO2_13K_113Lia1XFwd', '20250521_Janus Stacking SiO2_13K_113Lia1XBwd', '20250521_Janus Stacking SiO2_13K_113Lia1YFwd', '20250521_Janus Stacking SiO2_13K_113Lia1YBwd', '20250521_Janus Stacking SiO2_13K_113Lia1RFwd', '20250521_Janus Stacking SiO2_13K_113Lia1RBwd', '20250521_Janus Stacking SiO2_13K_113Lia2RFwd', '20250521_Janus Stacking SiO2_13K_113Lia2RBwd', '20250521_Janus Stacking SiO2_13K_113It_to_PCFwd', '20250521_Janus Stacking SiO2_13K_113It_to_PCBwd']
  cits: ['20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix']
  sts: []

🔍 調試：get_topo_files() 方法結果：
  ['20250521_Janus Stacking SiO2_13K_113TopoFwd', '20250521_Janus Stacking SiO

In [5]:
# 繪製 2D 拓撲圖 / Plot 2D Topography Map
if int_file_proxy:
    print(f"🎨 繪製 '{selected_int_file_key}' 的拓撲圖...")
    try:
        # 從 IntData 對象獲取圖像數據
        topo_data = int_file_proxy.data.image
        
        # 獲取物理尺寸用於座標軸
        x_range = int_file_proxy.data.x_range
        y_range = int_file_proxy.data.y_range
        physical_scale = (x_range, y_range)
        
        print(f"  使用物理尺寸: X={x_range:.2f} nm, Y={y_range:.2f} nm")

        fig_topo = SPMPlotting.plot_topography(
            image_data=topo_data,
            physical_scale=physical_scale,
            title=f"拓撲圖: {selected_int_file_key}",
            colorscale='RdYlBu_r' # 與 spm_plots.py 中的 HEIGHT_COLORSCALE 一致
        )
        fig_topo.show()
        print("✅ 拓撲圖繪製完成。")
    except Exception as e:
        print(f"❌ 拓撲圖繪製失敗: {e}")
        import traceback
        traceback.print_exc()
else:
    print("ℹ️  跳過拓撲圖繪製，因為沒有載入 .int 檔案。")

ℹ️  跳過拓撲圖繪製，因為沒有載入 .int 檔案。


In [6]:
# 提取並繪製高度剖面線 / Extract and Plot Height Profile
if int_file_proxy:
    print(f"📏 從 '{selected_int_file_key}' 提取剖面線: {PROFILE_START_COORD} → {PROFILE_END_COORD}")
    try:
        # 使用 int_file_proxy.analyzer.extract_line_profile 方法
        if hasattr(int_file_proxy, 'analyzer') and hasattr(int_file_proxy.analyzer, 'extract_line_profile'):
            profile_result = int_file_proxy.analyzer.extract_line_profile(
                PROFILE_START_COORD,  # start_point
                PROFILE_END_COORD,    # end_point
                'bresenham'           # method
            )
            
            if profile_result['success']:
                profile_data = profile_result['data']
                distances = profile_data['distance']
                heights = profile_data['height']  # 修正為 'height'
                
                # 獲取單位
                distance_unit = profile_data.get('distance_unit', 'nm')
                height_unit = profile_data.get('height_unit', 'nm')

                print(f"  ✅ 剖面線數據提取成功: {len(distances)} 個點")
                print(f"  📏 剖面線長度: {profile_data['length']:.2f} {distance_unit}")
                
                print("🎨 繪製高度剖面線...")
                fig_profile = SPMPlotting.plot_line_profile(
                    distances=distances,
                    heights=heights,
                    title=f"高度剖面線: {selected_int_file_key} ({PROFILE_START_COORD} → {PROFILE_END_COORD})",
                    x_unit=distance_unit,
                    y_unit=height_unit
                )
                fig_profile.show()
                print("✅ 高度剖面線繪製完成。")
            else:
                print(f"❌ 剖面線提取失敗: {profile_result.get('error', '未知錯誤')}")
        else:
            print("❌ 錯誤: `int_file_proxy.analyzer` 或 `extract_line_profile` 方法不存在。")
            print("  ℹ️  提示: 請確保 FileProxy 中已正確初始化 IntAnalyzer。")

    except Exception as e:
        print(f"❌ 高度剖面線處理失敗: {e}")
        import traceback
        traceback.print_exc()
else:
    print("ℹ️  跳過高度剖面線繪製，因為沒有載入 .int 檔案。")

ℹ️  跳過高度剖面線繪製，因為沒有載入 .int 檔案。


## 4. `.dat` (CITS) 檔案分析與繪圖 / `.dat` (CITS) File Analysis and Plotting

In [27]:
# 載入 CITS 數據 / Load CITS data
print(f"🔍 檢查可用的 CITS 檔案...")

# 先檢查可用的 CITS 檔案
available_cits = session.available_files['cits']
print(f"📋 可用的 CITS 檔案: {available_cits}")

# 如果設定的檔案不存在，使用第一個可用的 CITS 檔案
if CITS_FILE_KEY not in available_cits:
    if available_cits:
        CITS_FILE_KEY = available_cits[0]
        print(f"⚠️  原設定檔案不存在，改用: {CITS_FILE_KEY}")
    else:
        print("❌ 沒有找到任何 CITS 檔案")
        cits_file_proxy = None
        selected_cits_key = None
        cits_data_dict_for_analysis = None

if CITS_FILE_KEY in available_cits:
    print(f"🔍 載入 CITS 檔案: {CITS_FILE_KEY}")
    try:
        cits_file_proxy = session[CITS_FILE_KEY]
        cits_data_obj = cits_file_proxy.data
        selected_cits_key = CITS_FILE_KEY
        
        print(f"✅ CITS 數據載入成功")
        print(f"  檔案鍵值: {CITS_FILE_KEY}")
        print(f"  數據形狀: {cits_data_obj.shape}")
        print(f"  偏壓範圍: {cits_data_obj.bias_range[0]:.3f}V 到 {cits_data_obj.bias_range[1]:.3f}V")
        print(f"  偏壓點數: {cits_data_obj.n_bias_points}")
        print(f"  網格大小: {cits_data_obj.grid_size}")
        
        # 準備用於分析函數的數據格式 / Prepare data format for analysis functions
        cits_data_dict_for_analysis = {
            'data_3d': cits_data_obj.data_3d,
            'bias_values': cits_data_obj.bias_values,
            'grid_size': cits_data_obj.grid_size,
            'measurement_mode': 'CITS',
            'x_range': cits_data_obj.x_range,
            'y_range': cits_data_obj.y_range
        }
        print("  ✅ CITS 數據字典準備完成。")
        
    except Exception as e:
        print(f"❌ CITS 數據載入失敗: {e}")
        cits_file_proxy = None
        selected_cits_key = None
        cits_data_dict_for_analysis = None
else:
    cits_file_proxy = None
    selected_cits_key = None
    cits_data_dict_for_analysis = None

if cits_data_dict_for_analysis is None:
    print("🛑 無法繼續 CITS 檔案的分析，因為數據未成功準備。")
else:
    print(f"👍 準備使用 '{selected_cits_key}' 進行 CITS 檔案分析。")

🔍 檢查可用的 CITS 檔案...
📋 可用的 CITS 檔案: ['20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix']
🔍 載入 CITS 檔案: 20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix


/Users/yangziliang/Git-Projects/keen/backend/core/parsers/dat_parser.py:84: DtypeWarning:

Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.

檔案載入失敗: 20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix, 錯誤: '>=' not supported between instances of 'int' and 'str'


❌ CITS 數據載入失敗: Failed to load file '20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix': ["Failed to load file: '>=' not supported between instances of 'int' and 'str'"]
🛑 無法繼續 CITS 檔案的分析，因為數據未成功準備。


In [8]:
# 繪製 CITS 偏壓切片 / Plot CITS Bias Slice
if cits_data_dict_for_analysis:
    print(f"🔪 提取並繪製 CITS 偏壓切片，索引: {CITS_BIAS_INDEX}")
    try:
        # 檢查 BIAS_INDEX 是否在有效範圍內
        num_bias_points = len(cits_data_dict_for_analysis['bias_values'])
        if not (0 <= CITS_BIAS_INDEX < num_bias_points):
            print(f"⚠️  警告: BIAS_INDEX ({CITS_BIAS_INDEX}) 超出有效範圍 [0, {num_bias_points-1}]。將使用索引 0。")
            CITS_BIAS_INDEX = 0
            
        # 提取偏壓切片信息 (可選，主要為了獲取偏壓值用於標題)
        bias_slice_info = extract_cits_bias_slice(cits_data_dict_for_analysis, CITS_BIAS_INDEX)
        
        fig_cits_slice = SpectroscopyPlotting.plot_cits_bias_slice(
            data_3d=cits_data_dict_for_analysis['data_3d'],
            bias_values=cits_data_dict_for_analysis['bias_values'],
            bias_index=CITS_BIAS_INDEX,
            title=f"CITS 偏壓切片 @ {bias_slice_info['bias_value']:.3f}V (檔案: {selected_cits_key})",
            colorscale='Viridis',
            width=700,
            height=700
        )
        fig_cits_slice.show()
        print("✅ CITS 偏壓切片繪製完成。")
    except Exception as e:
        print(f"❌ CITS 偏壓切片繪製失敗: {e}")
else:
    print("ℹ️  跳過 CITS 偏壓切片繪製，因為數據未載入。")

ℹ️  跳過 CITS 偏壓切片繪製，因為數據未載入。


In [9]:
# 提取線剖面光譜數據並繪製能帶圖 / Extract Line Profile Spectra Data and Plot Band Diagram
if cits_data_dict_for_analysis:
    print(f"📏 提取 CITS 線剖面: {CITS_LINE_START} → {CITS_LINE_END}")
    try:
        cits_line_data = extract_line_spectra_data(
            cits_data_dict_for_analysis, 
            CITS_LINE_START, 
            CITS_LINE_END, 
            sampling_method='bresenham'
        )
        print(f"  ✅ CITS 線剖面提取成功: {cits_line_data['line_spectra'].shape[0]} 個點")

        print("🎨 繪製 CITS 能帶圖...")
        fig_cits_band = SpectroscopyPlotting.plot_band_diagram(
            line_spectra=cits_line_data['line_spectra'],
            bias_values=cits_line_data['bias_values'],
            distances=cits_line_data['distances'],
            title=f"CITS 能帶圖 - 線剖面 {CITS_LINE_START}→{CITS_LINE_END} (檔案: {selected_cits_key})",
            use_log_scale=CITS_USE_LOG_SCALE,
            colorscale='Viridis',
            width=900,
            height=600
        )
        fig_cits_band.show()
        print("✅ CITS 能帶圖繪製完成。")
        
    except Exception as e:
        print(f"❌ CITS 能帶圖處理失敗: {e}")
else:
    print("ℹ️  跳過 CITS 能帶圖繪製，因為數據未載入。")

ℹ️  跳過 CITS 能帶圖繪製，因為數據未載入。


In [10]:
# 準備並繪製堆疊光譜圖 / Prepare and Plot Stacked Spectra
if cits_data_dict_for_analysis and 'cits_line_data' in locals(): # 確保 cits_line_data 已成功提取
    print(f"📚 準備 CITS 堆疊光譜數據，最大曲線數: {CITS_MAX_CURVES}")
    try:
        cits_stacked_data = prepare_stacked_spectra_data(
            line_spectra=cits_line_data['line_spectra'], # 使用上面提取的線剖面數據
            bias_values=cits_line_data['bias_values'],
            max_curves=CITS_MAX_CURVES,
            step_selection='uniform'
        )
        print(f"  ✅ CITS 堆疊數據準備完成: {cits_stacked_data['n_selected']} 條曲線")

        print("🎨 繪製 CITS 堆疊光譜圖...")
        fig_cits_stacked = SpectroscopyPlotting.plot_stacked_spectra(
            line_spectra=cits_stacked_data['selected_spectra'],
            bias_values=cits_stacked_data['bias_values'],
            offset_factor=CITS_OFFSET_FACTOR,
            positions=cits_stacked_data['selected_positions'], # 使用 prepare_stacked_spectra_data 返回的位置
            max_curves=CITS_MAX_CURVES, # 確保與準備時一致
            title=f"CITS 堆疊光譜圖 - 線剖面 {CITS_LINE_START}→{CITS_LINE_END} (檔案: {selected_cits_key})",
            width=900,
            height=700
        )
        fig_cits_stacked.show()
        print("✅ CITS 堆疊光譜圖繪製完成。")
        
    except Exception as e:
        print(f"❌ CITS 堆疊光譜圖處理失敗: {e}")
elif not cits_data_dict_for_analysis:
    print("ℹ️  跳過 CITS 堆疊光譜圖繪製，因為 CITS 數據未載入。")
else:
    print("ℹ️  跳過 CITS 堆疊光譜圖繪製，因為 CITS 線剖面數據 (cits_line_data) 未成功提取。")


ℹ️  跳過 CITS 堆疊光譜圖繪製，因為 CITS 數據未載入。


## 5. 總結 / Summary

**整合視覺化測試完成！/ Integrated Visualization Test Complete!**

此筆記本展示了以下功能：
This notebook demonstrated the following functionalities:

- 從 `.txt` 檔案成功載入實驗數據。 / Successfully loaded experiment data from a `.txt` file.
- **對於 `.int` 檔案 (以 `{selected_int_file_key if 'selected_int_file_key' in locals() and selected_int_file_key else 'N/A'}` 為例):** / **For `.int` file (e.g., `{selected_int_file_key if 'selected_int_file_key' in locals() and selected_int_file_key else 'N/A'}`):**
    - 繪製了 2D 拓撲圖。 / Plotted 2D topography map.
    - 提取並繪製了高度剖面線。 / Extracted and plotted height profile.
- **對於 `.dat` CITS 檔案 (以 `{selected_cits_key if 'selected_cits_key' in locals() and selected_cits_key else 'N/A'}` 為例):** / **For `.dat` CITS file (e.g., `{selected_cits_key if 'selected_cits_key' in locals() and selected_cits_key else 'N/A'}`):**
    - 繪製了 CITS 偏壓切片。 / Plotted CITS bias slice.
    - 繪製了線剖面能帶圖。 / Plotted line profile band diagram.
    - 繪製了堆疊光譜圖。 / Plotted stacked spectra.

請檢查以上所有圖表是否按預期顯示。您可以調整本筆記本開頭「設定區域」中的參數以測試不同的檔案和條件。
Please check if all plots are displayed as expected. You can adjust the parameters in the "Configuration Section" at the beginning of this notebook to test with different files and conditions.